In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.model_selection import KFold
from tensorflow.keras.layers import Dense, InputLayer
from tensorflow.keras.models import Sequential

class TagHolder:

    def __init__(self, _id, name):
        self.id = _id
        self.name = name

    def __str__(self):
        return self.name


def get_tags():
    connection = get_engine()
    if not connection:
        return []

    try:
        with connection.cursor() as cursor:
            cursor.execute("SELECT * from datasetTag")
            tags = cursor.fetchall()

            tag_holder = []
            for tag in tags:
                tag_holder.append(TagHolder(tag[0], tag[1]))

            return tag_holder
    except Exception as e:
        return []
    finally:
        if connection.is_connected():
            connection.close()
            
            
from typing import Union

import mysql.connector
from mysql.connector.abstracts import MySQLConnectionAbstract
from mysql.connector.pooling import PooledMySQLConnection
from sqlalchemy.ext.declarative import declarative_base

Base = declarative_base()
def get_engine():
    # Create a MySQL connection using mysql.connector
    connection = mysql.connector.connect(
        host='monorail.proxy.rlwy.net',
        port=45826,
        user='root',
        password='VoUeejgBIkMgYiPmYHxMFsIXffwxCKBK',
        database='railway'
    )
    return connection





/Users/rufelleemmanuelpactol/InternshipData/SummerOJT/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/var/folders/m3/0c2cdk9j3lvc81rygncqrfy40000gn/T/ipykernel_28793/196199278.py:47: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


In [5]:


def build_model(input_shape, num_classes=3):
    model = Sequential([
        InputLayer(input_shape=(input_shape,)),
        Dense(160, activation='relu'),
        Dense(160, activation='relu'),
        Dense(160, activation='relu'),
        Dense(160, activation='relu'),
        Dense(160, activation='relu'),
        Dense(160, activation='relu'),
        Dense(160, activation='relu'),
        Dense(160, activation='relu'),
        Dense(160, activation='relu'),
        Dense(160, activation='relu'),
        Dense(160, activation='relu'),
        Dense(160, activation='relu'),
        Dense(160, activation='relu'),
        Dense(160, activation='relu'),
        Dense(160, activation='relu'),
        Dense(160, activation='relu'),
        Dense(num_classes, activation='softmax')
    ])
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

def train_nn(df: pd.DataFrame, k=5):
    import tensorflow as tf
    devices = tf.config.list_physical_devices()
    old_grades = df.weighted
    df['weighted'] = df.weighted.apply(discretize_weights)

    num_classes = 3

    # smote = SMOTE(random_state=42)
    X_resampled, y_resampled = (df.drop(columns=['weighted']), df['weighted'])

    X_resampled = X_resampled.values
    y_resampled = y_resampled.values

    kf = KFold(n_splits=k, shuffle=True, random_state=42)
    fold = 1
    all_histories = []
    all_evaluations = []

    for train_index, test_index in kf.split(X_resampled):
        X_train, X_test = X_resampled[train_index], X_resampled[test_index]
        y_train, y_test = y_resampled[train_index], y_resampled[test_index]

        model = build_model(X_train.shape[1], num_classes)
        early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)


        history = model.fit(X_train, y_train, epochs=300, batch_size=15, validation_data=(X_test, y_test), callbacks=[early_stopping], verbose=1)
        evaluation = model.evaluate(X_test, y_test)

        all_histories.append(history)
        all_evaluations.append(evaluation)

        ##st.write(f"Validation Fold {fold} Evaluation: (Log Loss: {evaluation[0]}, Accuracy: {evaluation[1]})")
        fold += 1

    avg_evaluation = np.mean(all_evaluations, axis=0)


    plt.figure(figsize=(10, 6))
    plt.plot(history.history['accuracy'], label='Train Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title('Model Accuracy (Last Fold)')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.ylim([0, 1])
    plt.legend(loc='upper right')
    plt.grid(True)
    ##st.pyplot(plt)

    accuracy_diff = np.array(history.history['accuracy']) - np.array(history.history['val_accuracy'])

    plt.figure(figsize=(10, 6))
    plt.plot(accuracy_diff, label='Train - Validation Accuracy Difference')
    plt.title('Train - Validation Accuracy Difference (Last Fold)')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy Difference')
    plt.legend(loc='upper right')
    plt.grid(True)
    ##st.pyplot(plt)

    w_df = pd.DataFrame(model.predict(pd.DataFrame(df.drop(columns=['weighted']))))
    w_df['original'] = (old_grades / 100) * 5
    w_df = w_df.rename(columns={0: 'Confidence to Fail', 1: 'Confidence to Pass', 2: 'Confidence to Excel'})

    return model, model.evaluate(X_test, y_test)


In [7]:
def discretize_weights(weight):
    if weight < 50:
        return 0
    if 50 <= weight < 85:
        return 1
    if 85 <= weight <= 100:
        return 2

In [8]:
def fetch_data_as_dataframe(connection, query: str) -> pd.DataFrame:
    cursor = connection.cursor()
    cursor.execute(query)
    result_set = cursor.fetchall()
    column_names = cursor.column_names
    cursor.close()
    df = pd.DataFrame(result_set, columns=column_names)
    return df.dropna()

In [6]:
def discretize_weights(weight):
    if weight < 50:
        return 0
    if 50 <= weight < 85:
        return 1
    if 85 <= weight <= 100:
        return 2

In [9]:
def ModelTrainingComponent():
    #st.header("Model Training Pipeline 🚀")
    #st.markdown("This section is for retraining and training models according to loaded datasets.")
    tags = get_tags()

    gen_set = '(5)'
    query = f"""
                    SELECT  (grade/5)*100 as weighted, attr_A, attr_B, attr_C, attr_E, attr_F, attr_H, attr_G, attr_I, attr_L, attr_M, attr_N, attr_O, attr_Q1, attr_Q2,
                            attr_Q3, attr_Q4, attr_EX, attr_AX, attr_TM, attr_IN, attr_SC, IF(cfit = 'L', 2,
                            IF(cfit = 'BA', 4,
                            IF(cfit = 'A', 6, 
                            IF(cfit = 'AA', 8,
                            IF(cfit = 'H', 10, NULL))))) as cfit, CASE when course = 'BSCS' then 1 else 0 end as course
                    FROM students
                    INNER JOIN assessments s on s.student_id = students.Id WHERE tagID in {gen_set};
                    """

    df = fetch_data_as_dataframe(get_engine(), query)
    model, result = train_nn(df)

    log_loss = result[0]
    accuracy = result[1]

    model.save('deep-learning-model.keras')
            #st.info(f"Finished training model, with log loss of {log_loss:.4f}, accuracy of {round(accuracy * 100, 2)}%.")

In [ ]:
ModelTrainingComponent()

2024-08-01 19:35:15.584 
  command:

    streamlit run /Users/rufelleemmanuelpactol/InternshipData/SummerOJT/.venv/lib/python3.9/site-packages/ipykernel_launcher.py [ARGUMENTS]
2024-08-01 19:35:15.585 No runtime found, using MemoryCacheStorageManager


Epoch 1/300


/Users/rufelleemmanuelpactol/InternshipData/SummerOJT/.venv/lib/python3.9/site-packages/keras/src/layers/core/input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(
